# Imports

In [53]:
%%capture
import pandas as pd 
import re
from pathlib import Path
from docx import Document
from datasets import load_dataset
from importlib import reload
from utils.eval import send_openrouter_request, models


In [30]:
# load transcripts
dir = Path('../data/transcripts')
doc_paths = sorted(dir.glob('*.docx'))

# Eval
"Common sense" checks using openrouter

In [31]:
# create string representing a single conversation
doc = Document(doc_paths[0])

paragraphs = []
for d in doc.paragraphs:
    # print(d.text)
    paragraphs.append(d.text)

conversation = ' '.join(paragraphs)

In [ ]:
# eval.
output = []
for m in models.values():
    print(f'evaluating {m}')
    messages = [{'role': 'user', 'content': f'name the speakers in this conversation, and give 1 sentence describing their role: {conversation}'}]
    response, reasoning, refusal, provider = send_openrouter_request(messages, model = m)
    output.append({'model': m, 'response': response, 'reasoning': reasoning, 'refusal': refusal, 'provider': provider})


evaluating qwen/qwen3-8B
evaluating qwen/qwen3-14B
evaluating qwen/qwen3-32B
evaluating google/gemma-2-9b-it


In [85]:
task = '''This transcript contains speaker labels. Some labels refer to the same real person.

Tasks:
1) How many unique real participants are there?
2) Which speaker labels are duplicates of the same person?
3) For each duplicate, say which label it should be merged into.
4) Give one short sentence describing each real participant.'''


In [87]:
messages = [{'role': 'user', 'content': f'{task}: {conversation}'}]

response, reasoning, refusal, provider = send_openrouter_request(messages, model = 'allenai/olmo-3.1-32b-instruct')
print(response)

Here’s a detailed answer to your four tasks based on the provided transcript:

---

### 1) How many unique real participants are there?

There are **three unique real participants** in the transcript:

- **Jasmine** (the interviewer/reporter from NBC News)
- **Speaker 4 / Lynn** (the CEO and co-founder of Fireworks, also referred to as "Lynn" by Jasmine at the end)
- **Speaker 1** (a person present at the start of the transcript, possibly a colleague or assistant of Jasmine, but not identified by name)

However, based on the context and the end of the transcript, "Speaker 1" is likely Jasmine herself or a third participant who is not central to the main interview. The main conversation is between Jasmine and Lynn (Speaker 4). "Speaker 3" appears briefly at the very beginning but does not participate in the main discussion.

But to be precise from the transcript:
- **Jasmine** (clearly identified)
- **Speaker 4 / Lynn** (identified as the CEO of Fireworks, and called "Lynn" at the end)


In [76]:
reasoning

In [66]:
df = pd.DataFrame(output)

In [73]:
from pprint import pprint
pprint(df['response'][3], width = 200)

('## Analysis of the Conversation\n'
 '\n'
 'This conversation provides a fascinating glimpse into the current state of open-source AI, particularly within the context of large language models (LLMs). \n'
 '\n'
 '**Key Takeaways:**\n'
 '\n'
 '* **Open-source LLMs are gaining traction:** Lynn, CEO of Fireworks, highlights the increasing adoption of open-source models by both startups and established enterprises, citing their impressive '
 'performance and cost-effectiveness.\n'
 '* **The gap between open and closed models is shrinking:** Lynn argues that the quality of open-source models is rapidly catching up to closed models, driven by advancements in research and the '
 'collaborative nature of the open-source community.\n'
 '* **Fine-tuning is crucial:** While base models are becoming increasingly powerful, Lynn emphasizes the importance of fine-tuning for specific applications. This allows companies to tailor models '
 'to their unique needs and achieve optimal performance.\n'
 '* 

In [40]:
print(response, reasoning, provider)

Based on the transcript, there are five speakers involved in this conversation:

*   **Jasmine:** An AI reporter for NBC News who is conducting the interview about open-source models.
*   **Speaker 1:** A representative from the PR agency Six Eastern who is coordinating the interview for her client, Lynn.
*   **Lynn (labeled as Speaker 4):** The CEO and co-founder of Fireworks who is being interviewed about her company and the AI industry.
*   **Speaker 2:** A participant who speaks briefly at the beginning of the call while getting their notes organized.
*   **Speaker 3:** A participant who makes a humorous comment during the initial small talk before the interview begins. **Analyzing the Request**

I've started breaking down the user's request. My focus is on understanding the core needs: speaker identification and the count. I'm prioritizing the primary and secondary goals initially. The tertiary goal hasn't been defined yet, so I'll address that later.


**Refining Speaker Profiles

# Cleaning

Speaker 1  12:32:55  
Yes, I know I'm sorry about all the back and forth, but I Lynn's schedule can be crazy, so

Jasmine   12:33:00  
no worries. I can't even imagine

Speaker 1  12:33:04  
CEO things. I feel like every, every meeting that she could possibly be in gets put on her calendar, and then we have to prioritize.

Jasmine   12:33:11  
Yeah, no, you're doing work in the same No,

Speaker 1  12:33:16  
you are. There's like, too many PR people in the world.

Speaker 2  12:33:21  
Okay, cool. Let me just get my other notes. It's 75 Google document tabs,

Jasmine   12:33:31  
yeah, oh man, whenever we have interns or like, students, will be like, I want to shadow you and see what you do is like, you will see me open a Google document and panic and like, that's all you're gonna see.

Speaker 1  12:33:44  
Yeah, you want to shadow and see my 900 open tabs.

Speaker 3  12:33:51  
Like, I'm not sure what you guys think people like adults do, but like, it ain't that

Speaker 1  12:33:5